In [1]:
# Setting imports, hyperparameters and some others
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple

import torch
import torchvision
print(f"torch version: {torch.__version__}")
print(f"torchvision version: {torchvision.__version__}")

from torch import nn
from torch.utils.data import DataLoader

try:
  from torchinfo import summary
except:
  print("Couldn't find torchinfo... installing it.")
  !pip install -q torchinfo
  from torchinfo import summary

import os

# Hyperparameters
BATCH_SIZE = 32
SEED = 1161
LEARNING_RATE = 1e-3
NUM_EPOCH = 20

# Other set-up
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_default_device(device) ###
print(f"running on {device}.")

torch version: 2.5.1+cu121
torchvision version: 0.20.1+cu121
running on cuda.


In [2]:
# Prepare food-101-tiny image data
import requests
import zipfile
import urllib.request

from pathlib import Path

image_path = Path("data/food-101-tiny")
file_name = "food101tiny.zip"

if image_path.is_dir():
  print(f"Image directory exists: {image_path}. Continue with existing data.")
else:
  print(f"Image directory not found: {image_path}")
  print("Proceed to download food-101-tiny from Kaggle...")
  image_path.mkdir(parents=True, exist_ok=True)
  urllib.request.urlretrieve("https://www.kaggle.com/api/v1/datasets/download/msarmi9/food101tiny", file_name)
  with zipfile.ZipFile(file_name, "r") as zip_ref:
    zip_ref.extractall()
  print("Images created.")

Image directory exists: data/food-101-tiny. Continue with existing data.


In [3]:
# Create model
def create_effnetb2_model(num_classes: int=10,
                          seed: int=1161,
                          dropout: float=0.3):
  """Creates an EfficientNetB2 feature extractor model and transforms.

  Args:
      num_classes: number of classes in the classifier head.
          Defaults: 10 (size of food-101-tiny).
      seed: random seed value.
      dropout: dropout rate of the classifier head.

  Returns:
      model: EffNetB2 feature extractor model.
      transforms: EffNetB2 image transforms.
  """

  weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
  transforms = weights.transforms()
  model = torchvision.models.efficientnet_b2(weights=weights)
  in_feature_num = 1408

  # Freezes all layers in base model
  for param in model.parameters():
    param.requires_grad = False

  # Changes classifier head
  torch.manual_seed(seed)
  model.classifier = nn.Sequential(
      nn.Dropout(p=dropout, inplace=True),
      nn.Linear(in_features=in_feature_num, out_features=num_classes),
  )

  # Note that model not compiled or moved to a device
  return model, transforms

effnetb2, effnetb2_transforms = create_effnetb2_model(seed=SEED)

is_compile_model = False # slower if run on T4-GPU
if is_compile_model:
  !pip install -U --index-url https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/Triton-Nightly/pypi/simple/ triton-nightly # needed for torch.compile()
  effnetb2 = torch.compile(effnetb2)

effnetb2 = effnetb2.to(device)

optimizer = torch.optim.Adam(params=effnetb2.parameters(), lr=LEARNING_RATE)
loss_fn = torch.nn.CrossEntropyLoss()

In [4]:
# Prepare DataLoader
train_dir = image_path / "train"
test_dir = image_path / "valid"
train_data = torchvision.datasets.ImageFolder(train_dir, transform=effnetb2_transforms)
test_data = torchvision.datasets.ImageFolder(test_dir, transform=effnetb2_transforms)
class_names = train_data.classes

train_dataloader_effnetb2 = DataLoader(train_data,
                                       batch_size=BATCH_SIZE,
                                       shuffle=True,
                                       generator=torch.Generator(device=device),
                                       )
test_dataloader_effnetb2 = DataLoader(test_data,
                                      batch_size=BATCH_SIZE,
                                      shuffle=False,
                                      generator=torch.Generator(device=device),
                                      )

In [5]:
%%writefile engine.py
"""
Contains functions for training and testing a PyTorch model.
"""
import torch
from tqdm.auto import tqdm
from typing import Dict, List, Tuple

def train_step(epoch: int,
               model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
  """Trains a PyTorch model for a single epoch

  Returns:
    A tuple containing train_loss (float), train_accuracy (float).
  """
  # Put model in train mode
  model.train()

  # Setup train loss and train accuracy values
  train_loss, train_acc = 0, 0

  # Loop through data loader data batches
  progress_bar = tqdm(
      enumerate(dataloader),
      desc=f"Training Epoch {epoch}",
      total=len(dataloader)
      )

  # Loop through data loader data batches
  for batch, (X, y) in progress_bar:
    # Send data to target device
    X, y = X.to(device), y.to(device)

    # 1. Forward pass
    y_pred = model(X)

    # 2. Calculate  and accumulate loss
    loss = loss_fn(y_pred, y)
    train_loss += loss.item()

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backward
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    # Calculate and accumulate accuracy metric across all batches
    y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
    train_acc += (y_pred_class == y).sum().item()/len(y_pred)

    # Update progress bar
    progress_bar.set_postfix(
        {
            "train_loss": train_loss / (batch + 1),
            "train_acc": train_acc / (batch + 1),
            }
        )

  # Adjust metrics to get average loss and accuracy per batch
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)
  return train_loss, train_acc

def test_step(epoch: int,
              model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
  """Tests a PyTorch model for a single epoch.

  Returns:
    A tuple containing test_loss (float), test_accuracy (float).
  """
  # Put model in eval mode
  model.eval()

  # Setup test loss and test accuracy values
  test_loss, test_acc = 0, 0

  # Loop through data loader data batches
  progress_bar = tqdm(
      enumerate(dataloader),
      desc=f"Testing Epoch {epoch}",
      total=len(dataloader)
      )

  # Turn on inference context manager
  with torch.inference_mode():
    # Loop through DataLoader batches
    #for batch, (X, y) in enumerate(dataloader):
    for batch, (X, y) in progress_bar:
      # Send data to target device
      X, y = X.to(device), y.to(device)

      # 1. Forward pass
      test_pred_logits = model(X)

      # 2. Calculate and accumulate loss
      loss = loss_fn(test_pred_logits, y)
      test_loss += loss.item()

      # Calculate and accumulate accuracy
      test_pred_labels = test_pred_logits.argmax(dim=1)
      test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

      progress_bar.set_postfix(
          {
            "test_loss": test_loss / (batch + 1),
            "test_acc": test_acc / (batch + 1),
            }
        )

  # Adjust metrics to get average loss and accuracy per batch
  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  return test_loss, test_acc

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:
  """Trains and tests a PyTorch model.

  Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for
    each epoch.
  """
  # Create empty results dictionary
  results = {"train_loss": [],
      "train_acc": [],
      "test_loss": [],
      "test_acc": []
  }

  # Loop through training and testing steps for a number of epochs
  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(epoch=epoch,
                                       model=model,
                                       dataloader=train_dataloader,
                                       loss_fn=loss_fn,
                                       optimizer=optimizer,
                                       device=device)
    test_loss, test_acc = test_step(epoch=epoch,
                                    model=model,
                                    dataloader=test_dataloader,
                                    loss_fn=loss_fn,
                                    device=device)

      # Print out what's happening
    print(f"Epoch: {epoch+1} | "
    f"train_loss: {train_loss:.4f} | "
    f"train_acc: {train_acc:.4f} | "
    f"test_loss: {test_loss:.4f} | "
    f"test_acc: {test_acc:.4f}")

      # Update results dictionary
    results["train_loss"].append(train_loss)
    results["train_acc"].append(train_acc)
    results["test_loss"].append(test_loss)
    results["test_acc"].append(test_acc)

  # Return the filled results at the end of the epochs
  return results

Overwriting engine.py


In [6]:
# Train the model
import engine
effnetb2_results = engine.train(model=effnetb2,
                                train_dataloader=train_dataloader_effnetb2,
                                test_dataloader=test_dataloader_effnetb2,
                                epochs=NUM_EPOCH,
                                optimizer=optimizer,
                                loss_fn=loss_fn,
                                device=device)

  0%|          | 0/20 [00:00<?, ?it/s]

Training Epoch 0:   0%|          | 0/47 [00:00<?, ?it/s]

Testing Epoch 0:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.8877 | train_acc: 0.4425 | test_loss: 1.4704 | test_acc: 0.7348


Training Epoch 1:   0%|          | 0/47 [00:00<?, ?it/s]

...

Testing Epoch 18:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch: 19 | train_loss: 0.3944 | train_acc: 0.8948 | test_loss: 0.5793 | test_acc: 0.8383


Training Epoch 19:   0%|          | 0/47 [00:00<?, ?it/s]

Testing Epoch 19:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch: 20 | train_loss: 0.3729 | train_acc: 0.9102 | test_loss: 0.5679 | test_acc: 0.8449


In [7]:
# Save trained model
target_dir_path = Path("models")
target_dir_path.mkdir(parents=True, exist_ok=True)
model_name = "pretrained_effnetb2_food101tiny.pth"
model_save_path = target_dir_path / model_name
print(f"Saving model to: {model_save_path}")
torch.save(obj=effnetb2.state_dict(), f=model_save_path)

Saving model to: models/pretrained_effnetb2_food101tiny.pth


In [8]:
# Run prediction for test_data with the trained model
import pandas as pd
from PIL import Image
from timeit import default_timer as timer
from tqdm.auto import tqdm

# 1. Create a function to return a list of dictionaries with sample, truth label, prediction, prediction probability and prediction time
def pred_and_store(paths: List[Path],
                   model: torch.nn.Module,
                   transform: torchvision.transforms,
                   class_names: List[str],
                   device: str = "cuda" if torch.cuda.is_available() else "cpu") -> List[Dict]:

  # 2. Create an empty list to store prediction dictionaries
  pred_list = []

  # 3. Loop through target paths
  for path in tqdm(paths):

    # 4. Create empty dictionary to store prediction information for each sample
    pred_dict = {}

    # 5. Get the sample path and ground truth class name
    pred_dict["image_path"] = path
    class_name = path.parent.stem
    pred_dict["class_name"] = class_name

    # 6. Start the prediction timer
    start_time = timer()

    # 7. Open image path
    img = Image.open(path)

    # 8. Transform the image, add batch dimension and put image on target device
    transformed_image = transform(img).unsqueeze(0).to(device)

    # 9. Prepare model for inference by sending it to target device and turning on eval() mode
    model.to(device)
    model.eval()

    # 10. Get prediction probability, predicition label and prediction class
    with torch.inference_mode():
      pred_logit = model(transformed_image) # perform inference on target sample
      pred_prob = torch.softmax(pred_logit, dim=1) # turn logits into prediction probabilities
      pred_label = torch.argmax(pred_prob, dim=1) # turn prediction probabilities into prediction label
      pred_class = class_names[pred_label.cpu()] # hardcode prediction class to be on CPU

      # 11. Make sure things in the dictionary are on CPU (required for inspecting predictions later on)
      pred_dict["pred_prob"] = round(pred_prob.unsqueeze(0).max().cpu().item(), 4)
      pred_dict["pred_class"] = pred_class

      # 12. End the timer and calculate time per pred
      end_time = timer()
      pred_dict["time_for_pred"] = round(end_time-start_time, 4)

    # 13. Does the pred match the true label?
    pred_dict["correct"] = class_name == pred_class

    # 14. Add the dictionary to the list of preds
    pred_list.append(pred_dict)

  # 15. Return list of prediction dictionaries
  return pred_list

# Make predictions across test dataset with EffNetB2
test_data_paths = list(Path(test_dir).glob("*/*.jpg"))
effnetb2_test_pred_dicts = pred_and_store(paths=test_data_paths,
                                          model=effnetb2,
                                          transform=effnetb2_transforms,
                                          class_names=class_names,
                                          device="cpu") # make predictions on CPU

effnetb2_test_pred_df = pd.DataFrame(effnetb2_test_pred_dicts)
effnetb2_test_pred_df.correct.value_counts()

effnetb2_average_time_per_pred = round(effnetb2_test_pred_df.time_for_pred.mean(), 4)
print(f"Average time per prediction: {effnetb2_average_time_per_pred} s")

  0%|          | 0/500 [00:00<?, ?it/s]

Average time per prediction: 0.1292 s


In [9]:
# Run demo with Gradio
try:
  import gradio as gr
except:
  print("[INFO] Couldn't find gradio... installing it.")
  !pip install -q gradio
  import gradio as gr

print(f"gradio version: {gr.__version__}")

import random

def predict(img) -> Tuple[Dict, float]:
  """Transforms and performs a prediction on img and returns prediction and time taken.
  """
  # Start the timer
  start_time = timer()

  # Transform the target image and add a batch dimension
  img = effnetb2_transforms(img).unsqueeze(0)

  # Put model into evaluation mode and turn on inference mode
  effnetb2.eval()
  with torch.inference_mode():
    # Pass the transformed image through the model and turn the prediction logits into prediction probabilities
    pred_probs = torch.softmax(effnetb2(img), dim=1)

  # Create a prediction label and prediction probability dictionary for each prediction class (this is the required format for Gradio's output parameter)
  pred_labels_and_probs = {class_names[i]: float(pred_probs[0][i]) for i in range(len(class_names))}

  # Calculate the prediction time
  pred_time = round(timer() - start_time, 5)

  # Return the prediction dictionary and prediction time
  return pred_labels_and_probs, pred_time

title = "FoodVision (food-101-tiny)"
class_names_str = ", ".join(class_names)
description = f"An EfficientNetB2 feature extractor computer vision model to classify images of food. Classes: {class_names_str}."
article = "ref: [09. PyTorch Model Deployment](https://www.learnpytorch.io/09_pytorch_model_deployment/). Model trained with [food-101-tiny](https://www.kaggle.com/datasets/msarmi9/food101tiny)."
example_list = [str(filepath) for filepath in random.sample(test_data_paths, k=3)]

demo = gr.Interface(fn=predict,
                    inputs=gr.Image(type="pil"),
                    outputs=[gr.Label(num_top_classes=3, label="Predictions"),
                             gr.Number(label="Prediction time (s)")],
                    examples=example_list,
                    title=title,
                    description=description,
                    article=article)
demo.launch(debug=False, share=True)

[INFO] Couldn't find gradio... installing it.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.0 MB/s eta 0:00:00
gradio version: 5.12.0
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6fea0bf6435240121f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [10]:
# All the code below is for deployment to HuggingFace
# URL: https://huggingface.co/spaces/rapianyo/foodvision_food101tiny
import shutil
from pathlib import Path

# Create FoodVision mini demo path
foodvision_food101tiny_demo_path = Path("demos/foodvision_food101tiny/")
if foodvision_food101tiny_demo_path.exists():
  shutil.rmtree(foodvision_food101tiny_demo_path)
  foodvision_food101tiny_demo_path.mkdir(parents=True, exist_ok=True)

foodvision_food101tiny_examples_path = foodvision_food101tiny_demo_path / "examples"
foodvision_food101tiny_examples_path.mkdir(parents=True, exist_ok=True)

# Copy example_list to the examples directory
for example in example_list:
  example_path = Path(example)
  destination = foodvision_food101tiny_examples_path / example_path.name
  print(f"[INFO] Copying {example_path} to {destination}")
  shutil.copy2(src=example_path, dst=destination)

# Create a destination path for our target model
effnetb2_foodvision_food101tiny_model_destination = foodvision_food101tiny_demo_path / model_name

# Try to move the file
try:
  print(f"[INFO] Attempting to move {model_save_path} to {effnetb2_foodvision_food101tiny_model_destination}")

  # Move the model
  shutil.move(src=model_save_path,
              dst=effnetb2_foodvision_food101tiny_model_destination)

  print(f"[INFO] Model move complete.")

# If the model has already been moved, check if it exists
except:
  print(f"[INFO] No model found at {model_save_path}, perhaps its already been moved?")
  print(f"[INFO] Model exists at {effnetb2_foodvision_food101tiny_model_destination}: {effnetb2_foodvision_food101tiny_model_destination.exists()}")

[INFO] Copying data/food-101-tiny/valid/ice_cream/1609293.jpg to demos/foodvision_food101tiny/examples/1609293.jpg
[INFO] Copying data/food-101-tiny/valid/ice_cream/138570.jpg to demos/foodvision_food101tiny/examples/138570.jpg
[INFO] Copying data/food-101-tiny/valid/edamame/1380888.jpg to demos/foodvision_food101tiny/examples/1380888.jpg
[INFO] Attempting to move models/pretrained_effnetb2_food101tiny.pth to demos/foodvision_food101tiny/pretrained_effnetb2_food101tiny.pth
[INFO] Model move complete.


In [11]:
# write create_effnetb2_model (exactly the same as the one above)
%%writefile demos/foodvision_food101tiny/model.py
import torch
import torchvision

from torch import nn

def create_effnetb2_model(num_classes: int=10,
                          seed: int=1161,
                          dropout: float=0.3):
  """Creates an EfficientNetB2 feature extractor model and transforms.

  Args:
      num_classes: number of classes in the classifier head.
          Defaults: 10 (size of food-101-tiny).
      seed: random seed value.
      dropout: dropout rate of the classifier head.

  Returns:
      model: EffNetB2 feature extractor model.
      transforms: EffNetB2 image transforms.
  """

  weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
  transforms = weights.transforms()
  model = torchvision.models.efficientnet_b2(weights=weights)
  in_feature_num = 1408

  # Freezes all layers in base model
  for param in model.parameters():
    param.requires_grad = False

  # Changes classifier head
  torch.manual_seed(seed)
  model.classifier = nn.Sequential(
      nn.Dropout(p=dropout, inplace=True),
      nn.Linear(in_features=in_feature_num, out_features=num_classes),
  )

  # model not compiled or moved to a device
  return model, transforms

Writing demos/foodvision_food101tiny/model.py


In [12]:
%%writefile demos/foodvision_food101tiny/app.py
import gradio as gr
import os
import torch

from model import create_effnetb2_model
from timeit import default_timer as timer
from typing import Tuple, Dict

SEED = 1161
class_names = ['apple_pie',
               'bibimbap',
               'cannoli',
               'edamame',
               'falafel',
               'french_toast',
               'ice_cream',
               'ramen',
               'sushi',
               'tiramisu'] # food101tiny_train_data.classes
class_names_str = ", ".join(class_names)
model_name = "pretrained_effnetb2_food101tiny.pth"
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_default_device(device)

effnetb2, effnetb2_transforms = create_effnetb2_model(seed=SEED)
effnetb2.load_state_dict(
    torch.load(
        f=model_name,
        map_location=torch.device(device),
    )
)
effnetb2 = effnetb2.to(device)

# Create predict function
def predict(img) -> Tuple[Dict, float]:
  """Transforms and performs a prediction on img and returns prediction and time taken.
  """
  # Start the timer
  start_time = timer()

  # Transform the target image and add a batch dimension
  img = effnetb2_transforms(img).unsqueeze(0)

  # Put model into evaluation mode and turn on inference mode
  effnetb2.eval()
  with torch.inference_mode():
    # Pass the transformed image through the model and turn the prediction logits into prediction probabilities
    pred_probs = torch.softmax(effnetb2(img), dim=1)

  # Create a prediction label and prediction probability dictionary for each prediction class (this is the required format for Gradio's output parameter)
  pred_labels_and_probs = {class_names[i]: float(pred_probs[0][i]) for i in range(len(class_names))}

  # Calculate the prediction time
  pred_time = round(timer() - start_time, 5)

  # Return the prediction dictionary and prediction time
  return pred_labels_and_probs, pred_time

### Create Gradio app ###
title = "FoodVision (food-101-tiny)"
description = f"An EfficientNetB2 feature extractor computer vision model to classify images of food. Classes: {class_names_str}."
article = "ref: [09. PyTorch Model Deployment](https://www.learnpytorch.io/09_pytorch_model_deployment/). Model trained with [food-101-tiny](https://www.kaggle.com/datasets/msarmi9/food101tiny)."
example_list = [["examples/" + example] for example in os.listdir("examples")]

demo = gr.Interface(fn=predict,
                    inputs=gr.Image(type="pil"),
                    outputs=[gr.Label(num_top_classes=3, label="Predictions"),
                             gr.Number(label="Prediction time (s)")],
                    examples=example_list,
                    title=title,
                    description=description,
                    article=article)
demo.launch(debug=False, share=True)

Writing demos/foodvision_food101tiny/app.py


In [13]:
%%writefile demos/foodvision_food101tiny/requirements.txt
torch
torchvision
gradio

Writing demos/foodvision_food101tiny/requirements.txt


In [14]:
# Download zip file - to be uploaded to HuggingFace
!cd demos/foodvision_food101tiny && zip -r ../foodvision_food101tiny.zip * -x "*.pyc" "*.ipynb" "*__pycache__*" "*ipynb_checkpoints*"
try:
    from google.colab import files
    files.download("demos/foodvision_food101tiny.zip")
except:
    print("Not running in Google Colab, can't use google.colab.files.download()")

  adding: app.py (deflated 56%)
  adding: examples/ (stored 0%)
  adding: examples/1609293.jpg (deflated 1%)
  adding: examples/1380888.jpg (deflated 0%)
  adding: examples/138570.jpg (deflated 0%)
  adding: model.py (deflated 53%)
  adding: pretrained_effnetb2_food101tiny.pth (deflated 8%)
  adding: requirements.txt (deflated 4%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>